In [1]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial import distance
import category_encoders as ce
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [2]:
def distace_to_centroid(row,centroids,colmns):
    centroid = centroids[row['CLUSTER_ID']]
    return distance.euclidean(centroid,row[colmns].values)

In [3]:
def load_data():
    #bond_df = pd.read_csv('bond_sample.csv')
    bond_df = pd.read_csv('bond_sample_30000.csv')
    snp_rtgs_df = pd.read_csv('snpRatings.csv')
    return {'bond_df':bond_df,'snp_rtgs_df':snp_rtgs_df}

In [4]:
def enrich_ratings(bond_df,snp_rtgs_df):
    orderedFeatures_Struct = {
                "B_RTG_SP":     dict(zip(snp_rtgs_df.snpMapped,snp_rtgs_df.Score)),                
               }
    bond_df=bond_df.replace(orderedFeatures_Struct)
    return bond_df
    

In [5]:
def feature_engineer(bond_df):
    cont_cols = ['DAYS_TO_MATURE','CPN','B_RTG_SP']
    cat_cols = ['STATE_CODE', 'BLOOMBERG_SECTOR_CODE', 'MUNI_ISSUE_TYP','FED_TAXABLE','CALLABLE','SINKABLE','PUTABLE']
    ct = ColumnTransformer(
        remainder='passthrough', #passthough features not listed
        transformers=[
            ('mm', MinMaxScaler() ,cont_cols),
            ('ce',ce.CountEncoder(),cat_cols),    
        ])
    scaled_data = ct.fit_transform(bond_df)
    reordered_colmns = [cont_cols,cat_cols,['CUSIP']]
    ordered_colmns = np.concatenate(reordered_colmns)
    bond_df_scaled = pd.DataFrame(scaled_data,columns=ordered_colmns.tolist())
    
    ct1 = ColumnTransformer(
        remainder='passthrough', #passthough features not listed
        transformers=[
             ('mm1', MinMaxScaler() ,cat_cols),            
     
        ])
    scaled_data1 = ct1.fit_transform(bond_df_scaled)
    reordered_colmns1 = [cat_cols,cont_cols,['CUSIP']]
    ordered_colmns1 = np.concatenate(reordered_colmns1)
    bond_df_scaled1 = pd.DataFrame(scaled_data1,columns=ordered_colmns1.tolist())
    bond_df_scaled = bond_df_scaled1[bond_df.columns]
    return bond_df_scaled

In [6]:
def fit_model(bond_df_scaled):
    bond_df_scaled_no_cusip =  bond_df_scaled[bond_df_scaled.columns[~bond_df_scaled.columns.isin(['CUSIP'])]]
    kmeans = KMeans(n_clusters=10,random_state=0).fit(bond_df_scaled_no_cusip)
    bond_df_scaled_no_cusip['CLUSTER_ID'] = kmeans.labels_
    colmns = bond_df_scaled_no_cusip[bond_df_scaled_no_cusip.columns[~bond_df_scaled_no_cusip.columns.isin(['CLUSTER_ID'])]].columns
    centroids = kmeans.cluster_centers_
    bond_df_scaled_no_cusip['DIST_TO_CENTROID'] = bond_df_scaled_no_cusip.apply(lambda row : distace_to_centroid(row,centroids,colmns),axis=1)
    bond_df_scaled_centroid_distance = bond_df_scaled[['CUSIP']].join(bond_df_scaled_no_cusip)
    bond_df_scaled_centroid_distance = bond_df_scaled_centroid_distance.sort_values(by=['CLUSTER_ID','DIST_TO_CENTROID'])
    return bond_df_scaled_centroid_distance

In [7]:
bond_df = load_data()['bond_df']
bond_df_original = bond_df
snp_rtgs_df = load_data()['snp_rtgs_df']
bond_df = enrich_ratings(bond_df,snp_rtgs_df)
bond_df_scaled = feature_engineer(bond_df)
bond_df_scaled_centroid_distance = fit_model(bond_df_scaled)
bond_df_scaled_centroid_distance[['CUSIP','CLUSTER_ID','DIST_TO_CENTROID']].join(bond_df_original,lsuffix='1',rsuffix='2')

CUSIP1  CLUSTER_ID  DIST_TO_CENTROID     CUSIP2  DAYS_TO_MATURE  \
12177  ICD6MFOQN           0          0.518107  ICD6MFOQN            7385   
4133   0YGZ87ONV           0          0.525369  0YGZ87ONV            7280   
15006  PBZQBBJRL           0          0.531906  PBZQBBJRL            7298   
4187   KKCL6PPNX           0          0.534427  KKCL6PPNX            6271   
25554  AC3E0546L           0          0.534847  AC3E0546L            7288   
...          ...         ...               ...        ...             ...   
14533  3TSECOL47           9          1.089740  3TSECOL47           12443   
3930   UBPSZJCYT           9          1.114506  UBPSZJCYT            1849   
5895   WQZ3V4BV0           9          1.121684  WQZ3V4BV0            2021   
1900   VS4MUAZBB           9          1.134795  VS4MUAZBB             556   
24342  H7X7DCU2S           9          1.141750  H7X7DCU2S             906   

        CPN B_RTG_SP STATE_CODE BLOOMBERG_SECTOR_CODE  \
12177  3.00       BB         ME                  JAIL   
4133   3.00        B         TN                  JAIL   
15006  3.00     BBB-         TN              SALESTAX   
4187   3.25       BB         NY                 PORTS   
25554  3.50        B         NJ                  JAIL   
...     ...      ...        ...                   ...   
14533  4.75      BB-         VT                  MISC   
3930   1.75        D         MN                  MISC   
5895   4.75       NR         UT                   SCD   
1900   1.75      AA+         MI                  MISC   
24342  1.50     BBB+         NH                  MISC   

                   MUNI_ISSUE_TYP  FED_TAXABLE  CALLABLE  SINKABLE  PUTABLE  
12177               REVENUE BONDS            1         0         0        1  
4133    CERTIFICATE PARTICIPATION            1         0         0        1  
15006   CERTIFICATE PARTICIPATION            1         0         0        1  
4187    CERTIFICATE PARTICIPATION            1         0         0        1  
25554               REVENUE BONDS            0         0         0        1  
...                           ...          ...       ...       ...      ...  
14533               REVENUE BONDS            1         0         1        0  
3930           SPECIAL OBLIGATION            1         0         1        0  
5895                REVENUE BONDS            1         0         1        0  
1900                REVENUE BONDS            1         0         1        0  
24342           GENERAL OBLIGATIO            1         0         1        0  

[30000 rows x 14 columns]

In [11]:
#!pip install faiss-cpu 

You should consider upgrading via the 'c:\installedsoftware\anaconda3\python.exe -m pip install --upgrade pip' command.


In [12]:
import faiss

In [14]:
bond_df_scaled_faiss = bond_df_scaled[bond_df_scaled.columns[~bond_df_scaled.columns.isin(['CUSIP'])]]
index = faiss.IndexFlatL2(bond_df_scaled_faiss.shape[1])
index.add(bond_df_scaled_faiss.astype(np.float32))
distances, neighbor_indices = index.search(bond_df_scaled_faiss.astype(np.float32), 6)
print(neighbor_indices[0],distances[0])
bond_df.iloc[neighbor_indices[0],:]

[    0  3743  6657 18840 25674  7902] [0.         0.01712513 0.04204178 0.06062508 0.0704999  0.07251453]


CUSIP  DAYS_TO_MATURE  CPN  B_RTG_SP STATE_CODE  \
0      4PH2198SZ            4706  4.5       120         KS   
3743   A861AMMG2            4884  4.5       140         MS   
6657   1F2YTCXH3            6266  4.5        80         CT   
18840  BHWWTZWYT            3305  4.0        80         FL   
25674  OR8WZCR1Y            3113  5.0       170         KS   
7902   UNTXBKPRN            2254  4.0       120         MA   

      BLOOMBERG_SECTOR_CODE             MUNI_ISSUE_TYP  FED_TAXABLE  CALLABLE  \
0                      SMEH         SPECIAL ASSESSMENT            0         0   
3743                 GARVEE   SPECIAL OBLIGATION BONDS            0         0   
6657                 PUBPWR     GENERAL OBLIGATION LTD            0         0   
18840                  SMEH         SPECIAL ASSESSMENT            0         0   
25674                GOHOSP     GENERAL OBLIGATION LTD            0         0   
7902               CMNTYCLG   GENERAL OBLIGATION UNLTD            0         0   

       SINKABLE  PUTABLE  
0             1        1  
3743          1        1  
6657          1        1  
18840         1        1  
25674         1        1  
7902          1        1